In [ ]:
def preprocessing(input_path):
  all_input_list = sorted(glob.glob(input_path))
  train = pd.DataFrame()
  for datapath in all_input_list:
    data = pd.read_csv(datapath) # 
    data['obs_time'] = data.index % 24 # 시간통일
    df = abs(data)
    df.loc[(df['내부온도관측치'] > 40), '내부온도관측치'] = 40
    df.loc[(df['내부습도관측치'] > 100), '내부습도관측치'] = 100
    df.loc[(df['co2관측치'] > 1200), 'co2관측치'] = 1200
    df.loc[(df['ec관측치'] > 8), 'ec관측치'] = 8
    df.loc[(df['시간당분무량'] > 3000), '시간당분무량'] = 3000
    df.loc[(df['시간당백색광량'] > 120000), '시간당백색광량'] = 120000
    df.loc[(df['시간당적색광량'] > 120000), '시간당적색광량'] = 120000
    df.loc[(df['시간당청색광량'] > 120000), '시간당청색광량'] = 120000
    df.loc[(df['시간당총광량'] > 120000), '시간당총광량'] = 120000
    col_list = df.columns
    for i in range(0,len(col_list)):
      col = col_list[i]    
      if '누적' in col :
        df[col] = df.groupby((df.obs_time == 0).cumsum()).agg(col_list[i-1]).cumsum()
    df.to_csv(datapath,index=False)
    train = pd.concat([train,df])
  print('finish!!')
  return train

In [ ]:
traininput = preprocessing('drive/MyDrive/dacon/data/train_input/*.csv')

In [ ]:
testinput = preprocessing('drive/MyDrive/dacon/data/test_input/*.csv')

In [ ]:
colname = traininput.columns

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
trainsd = scaler.fit(traininput.iloc[:,2:]).transform(traininput.iloc[:,2:])
testsd = scaler.fit(traininput.iloc[:,2:]).transform(testinput.iloc[:,2:])

trainsd = pd.DataFrame(trainsd,columns=colname[2:]).reset_index(drop=True)
testsd = pd.DataFrame(testsd,columns=colname[2:]).reset_index(drop=True)

train=pd.merge(traininput.iloc[:,:2].reset_index(drop=True),trainsd,left_index=True, right_index=True)
test=pd.merge(testinput.iloc[:,:2].reset_index(drop=True),testsd,left_index=True, right_index=True)

In [ ]:
def preprocessing2(input_path, data):
  all_input_list = sorted(glob.glob(input_path))
  for i in range(0,len(all_input_list)):
    df = data.iloc[ 24*28*i : 24*28*(i+1)]
    df.to_csv(all_input_list[i],index=False)

In [ ]:
preprocessing2('drive/MyDrive/dacon/data/train_input/*.csv',train)

In [ ]:
preprocessing2('drive/MyDrive/dacon/data/test_input/*.csv',test)

In [ ]:
def make_dataset(input_path, target_path):
    train = pd.DataFrame()
    all_input_list = sorted(glob.glob(input_path))
    all_target_list = sorted(glob.glob(target_path))
    for x, y in zip(all_input_list,all_target_list):
        x = pd.read_csv(x)
        y = pd.read_csv(y)
        x = x.iloc[:,1:]
        col_list = x.columns[1:]
        for i in range(0,28) :
            day = x.iloc[24*i:24*i+24]

            day_white = day[day['일간누적백색광량']>=0]['일간누적백색광량'].max()
            day_water = day[day['일간누적분무량']>=0]['일간누적분무량'].max()
            day_light = day[day['일간누적총광량']>=0]['일간누적총광량'].max()
            day_blue = day[day['일간누적청색광량']>=0]['일간누적청색광량'].max()
            day_red = day[day['일간누적적색광량']>=0]['일간누적적색광량'].max()

            day_white2 = day[day['일간누적백색광량']>=0]['일간누적백색광량'].mean()
            day_water2 = day[day['일간누적분무량']>=0]['일간누적분무량'].mean()
            day_light2 = day[day['일간누적총광량']>=0]['일간누적총광량'].mean()
            day_blue2 = day[day['일간누적청색광량']>=0]['일간누적청색광량'].mean()
            day_red2 = day[day['일간누적적색광량']>=0]['일간누적적색광량'].mean()

            time_list = day['obs_time'].unique()
            for col in col_list :
              if col in ['일간누적백색광량','일간누적총광량','일간누적청색광량','일간누적적색광량','일간누적분무량']:
                for t in range(0,len(time_list)):
                  time = time_list[t]
                  value1 = day[day['obs_time']==time][col].iloc[0]
                  x[col+str(time)] = value1
                  
              else :
                  for t in range(0,len(time_list)-3) :
                      time = time_list[t]
                      ntime = time_list[t+1]
                      nntime = time_list[t+2]
                      nnntime = time_list[t+3]
                      value1 = day[day['obs_time']==time][col].iloc[0]
                      value2 = day[day['obs_time']==ntime][col].iloc[0]
                      value3 = day[day['obs_time']==nntime][col].iloc[0]
                      value4 = day[day['obs_time']==nnntime][col].iloc[0]
                      x[col+str(time)+str("+")+str(nnntime)] = value1 + value2 + value3 + value4

            x['day_water_weight_max'] = (day_water*(i+1))
            x['day_white_weight_max'] = (day_white*(i+1))
            x['day_red_weight_max'] = (day_red*(i+1))
            x['day_blue_weight_max'] = (day_blue*(i+1))
            x['day_light_weight_max'] = (day_light*(i+1))

            x['day_water_weight_mean'] = (day_water2*(i+1))
            x['day_white_weight_mean'] = (day_white2*(i+1))
            x['day_red_weight_mean'] = (day_red2*(i+1))
            x['day_blue_weight_mean'] = (day_blue2*(i+1))
            x['day_light_weight_mean'] = (day_light2*(i+1))

            nx = x.iloc[:1,15:]
            ny = y.iloc[i:i+1].reset_index(drop=True)
            xy = pd.merge(nx,ny,left_index=True, right_index=True)
            train = pd.concat([train,xy]).reset_index(drop=True)

    return train

In [ ]:
train_input_path, train_target_path = 'drive/MyDrive/dacon/data/train_input/*.csv', 'drive/MyDrive/dacon/data/train_target/*.csv'
test_input_path, test_target_path = 'drive/MyDrive/dacon/data/test_input/*.csv', 'drive/MyDrive/dacon/data/test_target/*.csv'

train = make_dataset(train_input_path, train_target_path)
test = make_dataset(test_input_path, test_target_path)

In [ ]:
train=train.fillna(0)
test=test.fillna(0)

In [ ]:
train.to_csv('drive/MyDrive/dacon/data/train_standardscale_weight_max_mean_time4.csv',index=False)
test.to_csv('drive/MyDrive/dacon/data/test_standardscale_weight_max_mean_time4.csv',index=False)

In [ ]:
corr_df = train.corr()
corr_df = corr_df.apply(lambda x: round(x ,2))
s = corr_df.unstack()
성장 = pd.DataFrame(s['predicted_weight_g']).sort_values(by=0).fillna(0)
성장[0] = abs(성장[0])
성장.sort_values(by=[0],ascending=False).reset_index().to_csv('target값이랑corr.csv',index=False,encoding='utf-8-sig')
under_1per = 성장[성장[0]<0.15].index
under_1per

In [ ]:
drop_column = train.copy()
drop_column = drop_column.drop(under_1per,axis=1)

In [ ]:
drop_columntt = test.copy()
drop_columntt = drop_columntt.drop(under_1per,axis=1)

In [ ]:
import catboost
X = drop_column.drop(['predicted_weight_g'],axis=1)
y = drop_column['predicted_weight_g']
model= {}
for f in range(0,27):
      train_idx = [i for i in range(0,784)]
      valid_idx = train_idx[f*28:f*28+28]
      for idx in valid_idx:
        train_idx.remove(idx)
      print(f'===================================={f+1}============================================')
      #print(min(valid_idx),max(valid_idx))
      x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[valid_idx], y.iloc[train_idx], y.iloc[valid_idx]
      
      xgb_model = catboost.CatBoostRegressor( task_type='GPU',verbose=0,random_state=404)
      xgb_model.fit(x_train, y_train)
      
      y_pred = xgb_model.predict(x_val)
      rmse = mean_squared_error(y_val, y_pred)**0.5
      print(f"{f + 1} Fold rmse = {rmse}")
      model[f] = xgb_model
      print(f'================================================================================\n\n')

In [ ]:
testX = drop_columntt.drop(['predicted_weight_g'],axis=1)
pc=[0]*140
for fold in range(27):
    pc += model[fold].predict(testX)/27
pc

In [ ]:
i=0
all_target_list = sorted(glob.glob('drive/MyDrive/dacon/data/test_target/*.csv'))
for test_path in all_target_list:
    submit_df = pd.read_csv(test_path)
    submit_df['predicted_weight_g'] = pc[i*28:i*28+28]
    submit_df.to_csv(test_path, index=False)
    i+=1